## Importing the required packages

In [2]:
import ee
import geemap

from collections import defaultdict
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import ipywidgets as widgets
from ipywidgets import Layout
from pathlib import Path
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import json
import os

sns.set_theme()

In [ ]:
# Uncomment and run this cell if you are using Google Earth Engine (GEE) for the first time
# or if you encounter an "Earth Engine not initialized" error.

# ee.Authenticate()  # Run once to authenticate your Google account (only needed for new environments)
# ee.Initialize()    # Run every session to initialize the Earth Engine API

## 1. Data collection

### 1.1. Set the parameters

#### 1.1.1. Loading the variables for selection

In [3]:
# Dropdown widget configuration
parks = [
    'Addo Elephant', 'Agulhas', 'Augrabies Falls', 'Bontebok', 'Camdeboo', 'Garden Route',
    'Golden Gate Highlands', 'Graspan', 'Groenkloof', 'Kalahari Gemsbok', 'Karoo', 'Kruger',
    'Mapungubwe', 'Marakele', 'Mokala', 'Mountain Zebra', 'Namaqua', 'Richtersveld',
    'Table Mountain', 'Tankwa Karoo', 'West Coast'
]

years = [str(y) for y in range(2016, 2024)]

# Define dropdowns
park_dropdown = widgets.Dropdown(options=parks, value='Addo Elephant', description='Park:')
year_start_dropdown = widgets.Dropdown(options=years, value='2016', description='Start Year:')
year_end_dropdown = widgets.Dropdown(options=years, value='2023', description='End Year:')

# Store selections in a dictionary instead of global variables
selection = {
    "Park": park_dropdown.value,
    "Starting Year": int(year_start_dropdown.value),
    "Ending Year": int(year_end_dropdown.value)
}

# Generic update function
def update_selection(change, key):
    selection[key] = int(change.new) if key != "Park" else change.new

# Attach event listeners
park_dropdown.observe(lambda change: update_selection(change, "Park"), names='value')
year_start_dropdown.observe(lambda change: update_selection(change, "Starting Year"), names='value')
year_end_dropdown.observe(lambda change: update_selection(change, "Ending Year"), names='value')

#### 1.1.2. Variable parameters (Please select from the list below)

In [4]:
# Please choose parameters from the dropdown list below
display(park_dropdown, year_start_dropdown, year_end_dropdown)

Dropdown(description='Park:', options=('Addo Elephant', 'Agulhas', 'Augrabies Falls', 'Bontebok', 'Camdeboo', …

Dropdown(description='Start Year:', options=('2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'), …

Dropdown(description='End Year:', index=7, options=('2016', '2017', '2018', '2019', '2020', '2021', '2022', '2…

#### 1.1.3. Static parameters (Based on selection)

In [ ]:
# No need to set any variables
# Calculate the list of years from starting_year to ending_year, inclusive
Years = list(range(starting_year, ending_year + 1))

#Setting the properties for the layer to represent the data on the map and for color coding later on
dw_vis = {"min": 0, "max": 8, "palette": ["#419BDF", "#397D49", "#88B053", "#7A87C6", "#E49635", "#DFC35A", "#C4281B", "#A59B8F", "#B39FE1"]}

# Define class labels
class_labels = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare_soil', 'snow_and_ice']

# Load configuration settings
config_path = Path("config.json")  # Adjust for notebooks
with open(config_path, "r") as f:
    config = json.load(f)

# Set base path dynamically from config.json
base_path = Path(config["base_path"])

# Defining the different sub-areas
# CPA: Catchment Protected Area, VPA: Viewshed Protected Area, PNA: Priority Natural Areas, Parks: Park boundaries itself
potential_sub_areas = ['CPA', 'VPA', 'PNA', 'Parks', 'Dissolved']

# Initialize a list to store only the sub-areas with available shapefiles
sub_areas = []

# Check each sub-area for an available shapefile
for sub_area in potential_sub_areas:
    shapefile_path = os.path.join(base_path, sub_area, f'{Park}_{sub_area}.shp')
    if os.path.exists(shapefile_path):
        sub_areas.append(sub_area)

### 1.2. Initiate the map (set credentials)

In [ ]:
# This will prompt you to authorize via Earth Engine
# Data plotted later in notebook will be displayed on this map
Map = geemap.Map()
Map

### 1.3. Loading the data (server-side) according to above set parameters

### 1.3.1. Loading the data in Earth Engine

In [ ]:
# Initialize an empty dictionary to store results per sub-area and per year
results_per_area_and_year = {}

# Assuming 'fishnet' is already created as per the previous code snippet
for sub_area in sub_areas:
    # Update the path based on the new structure and sub-area
    park_sub_shp = fr'C:\Users\grobler\Desktop\Personal\Masters\Data\GIS\Parks_buffer\sub_areas\{sub_area}\{Park}_{sub_area}.shp'
    park_sub = geemap.shp_to_ee(park_sub_shp)
    geometry = park_sub.geometry()    

    # Initialize a dictionary for this sub-area to store dw_class objects per year
    dw_classes_per_year = {}

    for Year in Years:
        start_date = f'{Year}-01-01'
        end_date = f'{Year}-12-31'

        # This loads the DW dataset according to parameters set above
        dw_classes = geemap.dynamic_world(geometry, start_date, end_date, return_type='class', reducer='mode')
        dw_class = dw_classes.clip(geometry)

        # Store the dw_class in the dictionary with the year as the key
        dw_classes_per_year[Year] = dw_class

    # Store the results for this sub-area
    results_per_area_and_year[sub_area] = dw_classes_per_year

### 1.3.2. Plotting the data on the map

In [ ]:
# Please see the map above for results
# Recommended to plot individual samples using the following logic:
Map.addLayer(results_per_area_and_year['Dissolved'][2016], dw_vis, 'Exmaple LULC plot', False)

# Warning! This step takes long (use only when required)
#for sub_area, years_data in results_per_area_and_year.items():
#    for year, dw_class in years_data.items():
#        layer_name = f'DW {sub_area} for {year}'
#        Map.addLayer(dw_class, dw_vis, layer_name, False)

### 1.4. Creating the "Fisnet" (avoid API overload)

#### 1.4.2.1. Load dissolved buffers to get entire extent

In [ ]:
#Load the dissolved park AND buffer file
parks_shp = r'C:\Users\grobler\Desktop\Personal\Masters\Data\GIS\Parks_buffer\dissolved_all_buffers_FINAL.shp'
parks = geemap.shp_to_ee(parks_shp)
#Map.addLayer(parks, {}, 'dissolved_all_buffers_FINAL') #Can uncomment if you want to show all parks on the map above

#Some parks don't fall good in fishnet and should consider 
park_dissolved_shp = fr'C:\Users\grobler\Desktop\Personal\Masters\Data\GIS\Parks_buffer\sub_areas\Dissolved\{Park}_Dissolved.shp'
park_dissovled = geemap.shp_to_ee(park_sub_shp) 

#### 1.4.2.2. Create a "Fishnet" from above parameters

In [ ]:
# To apply the fishnet before processing large areas you have to determine the maximum extent
# Get the bounding box as a geometry object
bounding_box = parks.geometry().bounds()
#bounding_box = park_dissovled.geometry().bounds()

# Use getInfo() to convert the bounding box into a dictionary
bounding_box_info = bounding_box.getInfo()

# Extract the coordinates of the bounding box
# This accesses the first and only element of the coordinates list, which represents the bounding box polygon.
coords = bounding_box_info['coordinates'][0]  

# Extract min and max longitude and latitude
min_lon = coords[0][0]  # Minimum longitude
min_lat = coords[0][1]  # Minimum latitude
max_lon = coords[2][0]  # Maximum longitude
max_lat = coords[2][1]  # Maximum latitude

# Create a BBox object with these coordinates
region = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat)

# This creates the grid to overly the region/area of interest
fishnet = geemap.fishnet(region, h_interval=1.0, v_interval=1.0)

#### 1.4.2.3. Pre-calculate the sub-area and fishnet intersections (windows)

In [ ]:
# Initialize a dictionary to store window geometries for each sub-area
window_geometries_per_sub_area = {}

for sub_area in sub_areas:
    park_sub_shp = fr'C:\Users\grobler\Desktop\Personal\Masters\Data\GIS\Parks_buffer\sub_areas\{sub_area}\{Park}_{sub_area}.shp'
    park_sub = geemap.shp_to_ee(park_sub_shp)
    geometry = park_sub.geometry()
    
    # Calculate intersected features for this sub-area
    intersected_features = fishnet.map(lambda feature: ee.Feature(feature).intersection(geometry, ee.ErrorMargin(1)))
    
    # Initialize window geometries list
    window_geometries = []
    for feature in intersected_features.getInfo()['features']:
        geom = ee.Geometry(feature['geometry'])
        area = geom.area().getInfo()
        if area > 1:
            window_geometries.append(geom)
    
    # Store the window geometries for this sub-area
    window_geometries_per_sub_area[sub_area] = window_geometries

In [ ]:
#This is to plot an example of the overlapping Fishnet windows and sub-areas
# Define the style
#style = {"color": "ffff00ff", "fillColor": "00000000"}

# To convert the list to a feature collection for plotting
#geometries_list = window_geometries_per_sub_area['CPA'] # Please specify a sub-area of interest to you here
#features = [ee.Feature(geom) for geom in geometries_list]
#feature_collection = ee.FeatureCollection(features)

# To add the overlapping geometries of interest to the map
#Map.addLayer(feature_collection.style(**style), {}, "Window Geometries")

In [ ]:
#This is to plot an example of the overlapping Fishnet windows and sub-areas
# Define the style
#style = {"color": "ffff00ff", "fillColor": "00000000"}

# To convert the list to a feature collection for plotting
#geometries_list = window_geometries_per_sub_area['CPA'] # Please specify a sub-area of interest to you here
#features = [ee.Feature(geom) for geom in geometries_list]
#feature_collection = ee.FeatureCollection(features)

# To add the overlapping geometries of interest to the map
#Map.addLayer(fishnet.style(**style), {}, "Window Geometries")

### 1.5. Calculate the number of pixels per class

#### Note: This is the most time-consuming part of the script. But once all the data is derived the plotting can begin

In [ ]:
# Retrieve the LULC data per sub-area per year
for sub_area in sub_areas:
    # Access pre-calculated window geometries for the current sub-area
    window_geometries = window_geometries_per_sub_area[sub_area]

    all_years_data = []

    # Iterate through each year's data for the current sub-area
    for year, dw_class in tqdm(results_per_area_and_year[sub_area].items(), desc=f"Processing Years for {sub_area}"):
        aggregated_pixel_counts = defaultdict(int)

        # Iterate through each window geometry
        for window_geometry in tqdm(window_geometries, desc=f"Processing Windows for Year {year}"):
            # Perform analysis within this window
            pixel_count_stats = dw_class.reduceRegion(
                reducer=ee.Reducer.frequencyHistogram(),
                geometry=window_geometry,
                scale=10,  # Adjust the scale based on your dataset's resolution
                maxPixels=1e10
            ).getInfo()

            # Process the results directly
            pixel_counts = pixel_count_stats.get('label_mode', {})
            
            # Aggregate pixel counts across windows
            for key, count in pixel_counts.items():
                aggregated_pixel_counts[key] += count

        # Map numeric labels to actual class labels if necessary
        mapped_keys = {str(i): label for i, label in enumerate(class_labels)}  # Ensure class_labels is defined
        pixel_counts_formatted = {mapped_keys.get(key, key): value for key, value in aggregated_pixel_counts.items()}
        
        all_years_data.append({'Year': year, **pixel_counts_formatted})

    # After processing all years for the current sub-area, save to a CSV file
    df = pd.DataFrame(all_years_data)
    df = df.set_index('Year')
    filename = f'C:\\Users\\grobler\\Desktop\\Personal\\Masters\\Data\\DW_datasets\\{Park}\\{Park}_{sub_area}_LULC_from_{starting_year}_to_{ending_year}.csv'
    df.to_csv(filename, index=True)

## 3. Data collection of LULC CHANGES over time

### 3.1. Collecting & calculating the LULC change (server-side)

In [ ]:
# Initialize an empty dictionary to store LULC change results per sub-area and per year pair
results_per_area_and_year_pairs = {}

# Assuming 'sub_areas', 'Years', 'Park', and 'geometry' are already defined
for sub_area in sub_areas:
    # Update the path based on the new structure and sub-area
    park_sub_shp = fr'C:\\Users\\grobler\\Desktop\\Personal\\Masters\\Data\\GIS\\Parks_buffer\\sub_areas\\{sub_area}\\{Park}_{sub_area}.shp'
    # It seems there was a mistake in the variable used here; it should be 'park_sub_shp' instead of 'parks_shp'
    park_sub = geemap.shp_to_ee(park_sub_shp)
    geometry = park_sub.geometry()    

    # Initialize a dictionary for this sub-area to store dw_class objects per year
    dw_classes_per_year = {}

    for Year in Years:
        start_date = f'{Year}-01-01'
        end_date = f'{Year}-12-31'

        # This loads the DW dataset according to parameters set above
        dw_classes = geemap.dynamic_world(geometry, start_date, end_date, return_type='class', reducer='mode')
        dw_class = dw_classes.clip(geometry)

        # Store the dw_class in the dictionary with the year as the key
        dw_classes_per_year[Year] = dw_class

    # Now calculate LULC changes for each pair of consecutive years within this sub-area
    dw_classes_per_year_pairs = defaultdict(dict)
    for year_index in range(len(Years) - 1):
        pre_year = Years[year_index]
        post_year = Years[year_index + 1]

        image_pre = dw_classes_per_year[pre_year].select('label_mode')
        image_post = dw_classes_per_year[post_year].select('label_mode')

        # Combine the two images into a single image encoding transitions
        combined = image_pre.multiply(10).add(image_post)

        # Store the combined LULC change image for this year pair within the sub-area
        dw_classes_per_year_pairs[f"{pre_year}-{post_year}"] = combined

    # Store the LULC change results for this sub-area
    results_per_area_and_year_pairs[sub_area] = dw_classes_per_year_pairs

### 3.2. Populating a csv with the LULC change data

In [ ]:
for sub_area in sub_areas:
    # Access pre-calculated window geometries for the current sub-area
    window_geometries = window_geometries_per_sub_area[sub_area]

    yearly_transition_counts = {}

    # Iterate through each year pair's data for the current sub-area
    for year_pair, dw_class in tqdm(results_per_area_and_year_pairs[sub_area].items(), desc=f"Processing Year Pairs for {sub_area}"):
        year_pair_transition_counts = defaultdict(int)
        pre_year, post_year = year_pair.split('-')

        # Iterate through each window geometry
        for window_geometry in tqdm(window_geometries, desc=f"Processing Windows for Year Pair {year_pair}"):
            # Perform analysis within this window
            transition_counts = dw_class.reduceRegion(
                reducer=ee.Reducer.frequencyHistogram(),
                geometry=window_geometry,
                scale=10,
                maxPixels=1e10
            ).getInfo()

            transition_counts_dict = transition_counts.get('label_mode', {})

            # To create a dictionairy used later on for mapping
            transition_label_map = {}

            # Map combined values to transition labels and aggregate counts
            for i, label_pre in enumerate(class_labels):
                for j, label_post in enumerate(class_labels):
                    combined_value = str(i * 10 + j)
                    transition_label = f"{label_pre}_to_{label_post}"
                    count = transition_counts_dict.get(combined_value, 0)
                    year_pair_transition_counts[transition_label] += count
                    transition_label_map[combined_value] = transition_label

        # Store the aggregated counts for the current year transition
        yearly_transition_counts[f"{pre_year}_to_{post_year}"] = year_pair_transition_counts

    # Convert the dictionary to a DataFrame for CSV export
    df_transitions = pd.DataFrame(yearly_transition_counts).fillna(0).reset_index().rename(columns={'index': 'Change'})

    # Reordering columns
    ordered_cols = ['Change'] + sorted(df_transitions.columns[1:], key=lambda x: int(x.split('_to_')[0]))
    df_transitions = df_transitions[ordered_cols]

    # Output location and filename adjustment for each sub-area
    out_dir = fr'C:\Users\grobler\Desktop\Personal\Masters\Data\DW_datasets\{Park}'
    csv_file_path = os.path.join(out_dir, f'{Park}_{sub_area}_LULC_change_from_{starting_year}_to_{ending_year}.csv')

    # Export to CSV
    df_transitions.to_csv(csv_file_path, index=False)

# 2. Plotting the data collected above

### 2.1. Characterize the buffer zones

In [ ]:
# Load the dissolved buffer zones
dissolved_df = pd.read_csv(fr'C:\\Users\\grobler\\Desktop\\Personal\\Masters\\Data\\DW_datasets\\{Park}\\{Park}_Dissolved_LULC_from_{starting_year}_to_{ending_year}.csv', index_col='Year')
# Load the additional parks LULC data
parks_df = pd.read_csv(fr'C:\\Users\\grobler\\Desktop\\Personal\\Masters\\Data\\DW_datasets\\{Park}\\{Park}_Parks_LULC_from_{starting_year}_to_{ending_year}.csv', index_col='Year')

# Assuming 'dw_vis' contains your visualization settings including the palette
palette = dw_vis["palette"]

# Dynamically filter class_labels to only include columns that exist in dissolved_df or parks_df
valid_class_labels = [label for label in class_labels if label in dissolved_df.columns or label in parks_df.columns]

# Setup the figure size
fig, ax = plt.subplots(figsize=(15, 10))  # Width is now dependent on the number of years

# Plot the line graph for the dissolved buffer zones
for column in valid_class_labels:
    if column in dissolved_df.columns:  # Check if the column exists in dissolved_df
        color = palette[class_labels.index(column)]  # Use the index from class_labels to get the correct color
        ax.plot(dissolved_df.index, dissolved_df[column], marker='o', linestyle='-', label=column, color=color)

# Plot the line graph for the parks LULC with dashed lines
for column in valid_class_labels:
    if column in parks_df.columns:  # Check if the column exists in parks_df
        color = palette[class_labels.index(column)]  # Use the index from class_labels to get the correct color
        ax.plot(parks_df.index, parks_df[column], marker='o', linestyle='--', label=f'Park {column}', color=color)

# Customize axes and labels
ax.set_xlabel('Year')
ax.set_ylabel('Pixel count')
ax.set_yscale('log')
ax.set_title(f"LULC for {Park}'s park vs. dissolved buffer zone from {starting_year} to {ending_year}", fontsize=18)
ax.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xticks(dissolved_df.index)
ax.tick_params(axis='x', rotation=45)
ax.grid(True, which="both", ls="--")

# The relative size of the pie charts
relative_pie_size = 0.15  # Adjust this as needed for the size of the pie charts

# Use the x-ticks from the line graph to align the pie charts
xticks = ax.get_xticks()
xticks = xticks[(xticks >= dissolved_df.index.min()) & (xticks <= dissolved_df.index.max())]
xticks = xticks[:len(dissolved_df.index)]  # Ensure matching length

# Fill NaN values in 'dissolved_df' and 'parks_df' with 0
dissolved_df = dissolved_df.fillna(0)
parks_df = parks_df.fillna(0)

# Loop through the years and create pie charts
for i, year in enumerate(dissolved_df.index):
    # Get the position for the current year's pie chart
    pie_x = xticks[i]
    
    # Convert data coordinates (year) to figure fraction (0-1) coordinates
    trans = ax.transData + fig.transFigure.inverted()
    pie_x_fig, _ = trans.transform((pie_x, 0))
    
    # Define the pie chart rectangle
    pie_rect = [pie_x_fig - relative_pie_size / 2, 0.1, relative_pie_size, relative_pie_size]
    
    # Filter the data for this year, ensuring it matches the valid class labels
    pie_data = dissolved_df.loc[year, valid_class_labels]
    
    # Ensure the colors correspond to the correct classes
    pie_colors = [palette[class_labels.index(col)] for col in pie_data.index]

    # Create a new axis for the pie chart and plot it with percentages
    ax_pie = fig.add_axes(pie_rect, frameon=False)
    ax_pie.pie(pie_data, colors=pie_colors, startangle=90)

# Adjust the layout
plt.subplots_adjust(bottom=0.3)  # Adjust this as needed

# Show the plot
plt.show()

### 2.2. Plotting the normalized yearly difference

In [ ]:
# Calculate the differences between consecutive years
#dissolved_diff = dissolved_df.diff().dropna()
#parks_diff = parks_df.diff().dropna()
# Calculate the differences between consecutive years
dissolved_diff = dissolved_df.diff().fillna(0)
parks_diff = parks_df.diff().fillna(0)

# Normalize the differences to the maximum absolute value for each LULC class
dissolved_diff_normalized = dissolved_diff / dissolved_diff.abs().max()
parks_diff_normalized = parks_diff / parks_diff.abs().max()

# Filter only the classes that exist in both DataFrames
available_classes = [label for label in class_labels if label in dissolved_diff.columns and label in parks_diff.columns]

# Determine the number of available classes
num_classes = len(available_classes)

# Dynamically calculate the number of rows and columns based on the number of available classes
ncols = 3
nrows = (num_classes + ncols - 1) // ncols  # Calculate the number of rows to fit the available classes

# Create the subplots with the calculated number of rows and columns
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 12))
axes = axes.flatten()

for i, column in enumerate(available_classes):
    ax = axes[i]
    
    # Bar width and position adjustments
    width = 0.35
    x = np.arange(len(dissolved_diff.index))
    
    # Plot for dissolved buffer zones
    color = palette[class_labels.index(column)]  # Ensure color consistency
    ax.bar(x - width/2, dissolved_diff_normalized[column], width, label=f'Dissolved {column}', color=color)
    
    # Plot for parks with dashed edge
    ax.bar(x + width/2, parks_diff_normalized[column], width, label=f'Park {column}', color='none', edgecolor=color, linestyle='--', hatch='//')
    
    # Set axis properties
    ax.set_title(column)
    ax.set_xticks(x)
    ax.set_xticklabels(dissolved_diff.index, rotation=45)
    ax.set_ylim(-1.1, 1.1)  # Set y-axis limits to -1 and 1
    ax.legend()
    ax.grid(True, which="both", ls="--")

# Remove any empty subplots (if applicable)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle(f"Normalized yearly difference in LULC for {Park}'s park vs. dissolved buffer zone from {starting_year} to {ending_year}", fontsize=18)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 2.2. Plotting the LULCC on Sankey diagram

#### 2.2.1. Prepare the data in Sankey format

In [ ]:
# Load the dissolved buffer zones
dissolved_change_df = pd.read_csv(fr'C:\\Users\\grobler\\Desktop\\Personal\\Masters\\Data\\DW_datasets\\{Park}\\{Park}_Dissolved_LULC_change_from_{starting_year}_to_{ending_year}.csv', index_col='Change')

# Initialize lists to store the source, target, and values for the Sankey diagram
sources = []
targets = []
values = []

# Number of class labels
num_labels = len(class_labels)

# Generate sources, targets, and values from the DataFrame
for col in dissolved_change_df.columns:
    from_year, to_year = col.split('_to_')
    for index, value in dissolved_change_df[col].items():
        if value > 0:  # Only create a link if there's a non-zero value
            from_class, to_class = index.split('_to_')
            source_index = (int(from_year) - starting_year) * num_labels + class_labels.index(from_class)
            target_index = (int(to_year) - starting_year) * num_labels + class_labels.index(to_class)
            sources.append(source_index)
            targets.append(target_index)
            values.append(value)

# Define node labels and colors (repeated for each year)
node_labels = [f'{label}' for year in range(starting_year, ending_year + 1) for label in class_labels]
node_colors = palette * (ending_year - starting_year + 1)  # Ensure each node has a color

#### 2.2.2. Plot the Sankey-diagrams

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=node_labels,
        color=node_colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values
    ))])

# Calculate the proportional positions of the year annotations along the x-axis
year_positions = {year: (year - starting_year) / (ending_year - starting_year) for year in range(starting_year, ending_year + 1)}

# Update layout to add year annotations below the diagram
fig.update_layout(
    title=dict(
        text=f"LULC change for {Park}'s dissolved buffer zone from {starting_year} to {ending_year}",
        font=dict(size=20)  # Here you can adjust the size of the title
    ),
    font=dict(size=10),  # Adjusts the global font size used in the diagram
    annotations=[dict(
        showarrow=False,
        text=str(year),  # Year label
        xref="paper",
        yref="paper",
        x=year_positions[year],  # Updated x position for each year
        y=-0.1,
        align="center",
        font=dict(size=10)  # This adjusts the annotation text size
    ) for year in range(starting_year, ending_year + 1)],
    height=600  # Adjust the height as needed for your diagram
)

# Disable the x and y axes (since we have our custom annotations for years)
#fig.update_xaxes(visible=False)
#fig.update_yaxes(visible=False)

fig.show()

# 3. Calculate the LULCC intensity

### 3.1. Choose sub-area for further investigation

In [ ]:
# Define the Dropdown Widget
sub_area_dropdown = widgets.Dropdown(
    options=sub_areas,
    value='Dissolved',
    description='Buffer sub-area for further investigation:',
    disabled=False,
)

# Create a Variable
inv_sub_area = sub_area_dropdown.value

# Define an Update Function
def inv_sub_area_change(change):
    global inv_sub_area
    inv_sub_area = change.new
    print(f"Buffer sub-area to investigate changed to: {inv_sub_area}")

# Attach the Update Function to the Dropdown
sub_area_dropdown.observe(inv_sub_area_change, names='value')

#Print note to user
print('Note: Recommened to start with "Disolved" (standard setting) first and then move on to other options as deemed necessairy.\n\nCPA: Catchment Protected Area, VPA: Viewshed Protected Area, PNA: Priority Natural Areas, Parks: Park boundaries themselves')

# To display the dropdown in a Jupyter notebook
display(sub_area_dropdown)

### 3.2. Prepare data in "cross-tabulation matrix" format

In [ ]:
#Load the respective sub-area for further investigation
df = pd.read_csv(fr'C:\Users\grobler\Desktop\Personal\Masters\Data\DW_datasets\{Park}\{Park}_{sub_area_dropdown.value}_LULC_change_from_{starting_year}_to_{ending_year}.csv')

# The CSV has transitions labeled as 'from_to'. We need to split these into two separate columns for pivot.
df[['from', 'to']] = df['Change'].str.split('_to_', expand=True)

# Initialize an empty dictionary to hold the transition matrices
transition_matrices = {}

for year in df.columns[1:-2]:  # Skips the first column and the last two columns, assuming the first columns is 'Change' and the last two are 'from' and 'to'
    # Pivot the full table to create a matrix for the current year transition
    matrix = df.pivot(index='from', columns='to', values=year).fillna(0)

    # Since we're going to add this as a new row, we need to transpose the matrix first, add the row, then transpose back
    matrix = matrix.T
    # Calculate 'Final total' - sum of the columns in the matrix
    matrix['Final total'] = matrix.sum(axis=1)
    # Calculate 'Gross gain' - 'Final total' - class==class
    matrix['Gross gain'] = matrix['Final total'] - [matrix.at[state, state] if state in matrix.columns else 0 for state in matrix.index]
    #Flip the table back now that 'Final total' and 'Gross gain' was calculated
    matrix = matrix.T

    # Calculate 'Initial total' - sum of the rows in the matrix
    matrix['Initial total'] = matrix.sum(axis=1)
    # Calculate 'Gross gain' - 'Initial total' - class==class
    matrix['Gross loss'] = matrix['Initial total'] - [matrix.at[state, state] if state in matrix.columns else 0 for state in matrix.index]

    #Populate the dictionary
    transition_matrices[year] = matrix

### 3.3. Calcualte the "Time Intensity" of LULCC

#### 3.3.1. "Time Intensity" calculation (St & U)

In [ ]:
# Initialize lists to hold the calculated values
time_intervals = []
annual_rates_of_change = []

# Loop through each year interval
for year_interval, matrix in transition_matrices.items():
    # Retrieve the total area of change for the interval
    total_area_of_change = matrix['Gross loss']['Gross gain']
    # Retrieve the total area of the study area
    total_area_of_study_region = matrix['Initial total']['Final total']
    # Assume a duration of the interval in years of 1 for simplicity
    duration_of_interval = 1
    # Calculate the annual rate of change (time intensity)
    time_intensity = (total_area_of_change / total_area_of_study_region) / duration_of_interval * 100
    
    # Store the results
    time_intervals.append(year_interval)
    annual_rates_of_change.append(time_intensity)

# Calculate the uniform intensity value across all intervals
total_change_over_all_intervals = sum([matrix['Gross loss']['Gross gain'] for matrix in transition_matrices.values()])
#total_area_of_study_region_all_intervals = sum([matrix['Initial total']['Final total'] for matrix in transition_matrices.values()])
total_time = len(time_intervals)
uniform_intensity = (total_change_over_all_intervals / total_area_of_study_region) / total_time * 100

# Initialize significant_intervals to None
significant_intervals = None
# Only update significant_intervals if inv_sub_area is 'Dissolved'
if significant_intervals is None or inv_sub_area == 'Dissolved':
    # Find the years for which the annual_rates_of_change > uniform_intensity
    # This is to be used in all plots following this initial time intensity analysis
    significant_intervals = [year for year, rate in zip(time_intervals, annual_rates_of_change) if rate > uniform_intensity]

print(significant_intervals)

In [ ]:
#significant_intervals = ['2016_to_2017', '2021_to_2022']

#### 3.3.2. "Time Intensity" plots

In [ ]:
# Plot the results
plt.figure(figsize=(10, 5))

# Plot the annual rates of change
plt.barh(time_intervals, annual_rates_of_change, color='gray', edgecolor='black')

# Plot the uniform intensity value
plt.axvline(x=uniform_intensity, color='red', linestyle='--', label=f'{uniform_intensity:.2f} = Uniform Intensity')
# Add 'Slow' and 'Fast' labels relative to the uniform line
plt.text(uniform_intensity - 0.08, 5.7, 'Slow', verticalalignment='center', horizontalalignment='right', color='red', fontsize=12)
plt.text(uniform_intensity + 0.08, 5.7, 'Fast', verticalalignment='center', horizontalalignment='left', color='red', fontsize=12)

# Add labels and title
plt.xlabel('Annual Change Area (percent of map)')
plt.ylabel('Time interval')
plt.title(f"LULCC time intensity analysis for {Park}'s {inv_sub_area} buffer zone", fontsize=15)
plt.legend()

# Show the plot
plt.show()

### 3.4. Calcualte the "Category Intensity" of LULCC

#### 3.4.1. "Category Intensity" calculation (Gtj & Lti)

###### Note: the code is currently developed to only analyse those years with St > U indetified above

In [ ]:
# Initialize dictionary to hold category intensities for the significant intervals
category_intensities = {}

# For each significant year, calculate the category gain intensities
for year in significant_intervals:
    # Get the transition matrix for the current year
    matrix = transition_matrices[year]
    
    # Initialize a dictionary to store loss and gain intensities for the current year
    loss_intensities = {}
    gain_intensities = {}
    
    # Loop through the categories in the matrix
    for category in matrix.index[:-2]:  # Exclude 'Final total' and 'Gross gain' rows
        initial_total = matrix.at[category, 'Initial total']
        final_total = matrix.T.at[category, 'Final total']
        gross_loss = matrix.at[category, 'Gross loss']
        gross_gain = matrix.T.at[category, 'Gross gain']
        
        # Calculate loss intensity
        if initial_total > 0:
            loss_intensity = (gross_loss / duration_of_interval) / initial_total * 100
            loss_intensities[category] = loss_intensity
        
        # Calculate gain intensity
        if final_total > 0:
            gain_intensity = (gross_gain / duration_of_interval) / final_total * 100
            gain_intensities[category] = gain_intensity
    
    # Store the intensities for the current year
    category_intensities[year] = {
        'loss_intensities': loss_intensities,
        'gain_intensities': gain_intensities
    }

#### 3.4.2. "Category Intensity" plots

In [ ]:
# Define the total number of categories
total_categories = 9

# Loop over each significant interval to plot their data
for interval in significant_intervals:
    # Get the data for the current interval
    data = category_intensities[interval]

    # Extract existing categories, but fill in missing ones with 0s
    categories = list(data['loss_intensities'].keys())  # Get the list of available categories
    loss_intensities = [data['loss_intensities'].get(cat, 0) for cat in categories]
    gain_intensities = [data['gain_intensities'].get(cat, 0) for cat in categories]

    # Padding with zeros for intensities but leave labels empty
    empty_slots = total_categories - len(categories)
    loss_intensities += [0] * empty_slots
    gain_intensities += [0] * empty_slots
    # Pad categories with empty strings for the labels
    categories += [''] * empty_slots

    # Find the index of the interval to get the annual rate of change
    index = time_intervals.index(interval)
    # Get the specific uniform intensity for this interval
    specific_uniform_intensity = annual_rates_of_change[index]

    # Position of bars on the y-axis
    y_pos = np.arange(total_categories)
    
    # Create the plot
    plt.figure(figsize=(10, 5))

    # Create horizontal bars for losses
    plt.barh(y_pos, loss_intensities, color='tomato', edgecolor='black', height=0.4, label='Loss Intensity')

    # Create horizontal bars for gains, slightly offset on the y-axis
    plt.barh(y_pos + 0.4, gain_intensities, color='mediumseagreen', edgecolor='black', height=0.4, label='Gain Intensity')

    # Draw a dashed line for the uniform intensity
    plt.axvline(x=specific_uniform_intensity, color='black', linestyle='--', label=f'Uniform Intensity {specific_uniform_intensity:.2f}%')
    # Add 'Dormant' and 'Active' labels relative to the uniform line
    plt.text(specific_uniform_intensity - 0.25, +8.8, 'Dormant', verticalalignment='center', horizontalalignment='right', color='black', fontsize=10)
    plt.text(specific_uniform_intensity + 0.5, +8.8, 'Active', verticalalignment='center', horizontalalignment='left', color='black', fontsize=10)

    # Labels and title
    plt.xlabel('Annual Change Intensity (percent of category)')
    plt.title(f"LULCC category intensity analysis for {Park}'s {inv_sub_area} buffer zone from {interval}", fontsize=13)
    plt.yticks(y_pos + 0.2, categories)  # Set y-ticks to be in the middle of the two bars, leaving empty labels for missing categories
    plt.legend()

    # Show the plot
    plt.show()

In [ ]:
# This is purely for writing purposes (can copy this output into thesis)
# Prepare an empty list to collect data
#summary_data = []

# Loop over each significant interval for printing summaries in a table
#for interval in significant_intervals:
    # Get the data for the current interval
#    data = category_intensities[interval]
#    categories = list(data['loss_intensities'].keys())
#    loss_intensities = [data['loss_intensities'][cat] for cat in categories]
#    gain_intensities = [data['gain_intensities'].get(cat, 0) for cat in categories]

#    index = time_intervals.index(interval)
#    specific_uniform_intensity = annual_rates_of_change[index]

    # Collect data for the DataFrame only if the intensities are above the uniform intensity
#    for cat, loss, gain in zip(categories, loss_intensities, gain_intensities):
#        if loss > specific_uniform_intensity or gain > specific_uniform_intensity:
#            summary_data.append({
#                'Interval': interval,
#                'Category': cat,
#                'Loss Intensity (%)': loss,
#                'Gain Intensity (%)': gain,
#                'Uniform Intensity (%)': f"{specific_uniform_intensity:.2f}",
                #'Loss > Uniform': loss > specific_uniform_intensity,
                #'Gain > Uniform': gain > specific_uniform_intensity
#            })

# Create DataFrame
#summary_df = pd.DataFrame(summary_data)

# Display the DataFrame
#summary_df

### 3.5. Calcualte the "Transition Intensity" of LULCC (Qtmj, Vtm, Rtin and Wtn)

#### 3.5.1. "Transition Intensity" calculation for Qtmj, Vtm -> Loss, transition from target category to all other categories.

In [ ]:
# Initialize dictionary to hold category transition intensities for the significant intervals
category_transitions = {}

for year in significant_intervals:
    matrix = transition_matrices[year]

    # Correcting the approach to access 'Final total' from the rows, not the columns
    # Also, ensuring we sum the correct elements for 'total_area_non_m'
    final_totals = matrix.loc['Final total'][:-2]  # Assuming 'Final total' and 'Gross gain' are at the end
    total_area = final_totals.sum()

    # Initialize a matrix to store transition intensities for the current year, including a column for uniform intensity
    transition_intensities = pd.DataFrame(0, index=matrix.index[:-2], columns=matrix.columns[:-2].append(pd.Index(['Uniform_Intensity'])))

    for category_from in matrix.index[:-2]:  # Exclude 'Final total' and 'Gross gain' rows
        gross_loss = matrix.loc[category_from, 'Gross loss']
        area_not_m = total_area - matrix.loc['Final total', category_from]
        
        if area_not_m > 0:  # Avoid division by zero
            uniform_intensity = (gross_loss / duration_of_interval) / area_not_m * 100
            transition_intensities.loc[category_from, 'Uniform_Intensity'] = uniform_intensity

        for category_to in matrix.columns[:-2]:  # Exclude 'Initial total' and 'Gross loss' columns
            if category_from != category_to:  # Ensure transitions between different categories
                transition_area = matrix.loc[category_from, category_to]
                final_total_category_to = matrix.loc['Final total', category_to]
                
                if final_total_category_to > 0:  # Avoid division by zero
                    transition_intensity = (transition_area / duration_of_interval) / final_total_category_to * 100
                    transition_intensities.at[category_from, category_to] = transition_intensity

    category_transitions[year] = transition_intensities

#### 3.5.2. "Transition Intensity" plots for Qtmj, Vtm -> Loss, transition from target category to all other categories.

In [ ]:
for year in significant_intervals:
    matrix_to_plot = category_transitions[year]
    
    # Rounding the matrix values
    matrix_to_plot = matrix_to_plot.round(0)
    
    # Update the mask for the new shape
    mask = np.zeros_like(matrix_to_plot, dtype=bool)
    np.fill_diagonal(mask, True)  # Continue to hide diagonal values

    # Setting the colors for the mask
    cmap = mpl.colormaps.get_cmap('Greens')
    cmap.set_bad("white")  # setting background for masked elements
    
    # Plot heatmap with updated settings
    plt.figure(figsize=(12, 9))
    ax = sns.heatmap(matrix_to_plot, annot=True, fmt=".0f", cmap=cmap, linewidths=.5, mask=mask)
    
    # Add vertical lines
    for i in range(matrix_to_plot.shape[1] + 1):  # adding lines at each column edge
        plt.axvline(x=i, color='black', linestyle='-', linewidth=1)
    
    plt.title(f"LULC transition intensity analysis for {Park}'s {inv_sub_area} buffer zone from {year}", fontsize=14)
    plt.xlabel('To Category & Uniform Intensity (percent of category)')
    plt.xticks(rotation=45, ha="right")
    plt.ylabel('From Category (percent of category)')
    
    plt.show()

In [ ]:
######################################Clarify in the hadings the difference between Rtin and Qtmj#################################################
#From trees to flooded vegetation
#Qtmj=((matrix)/duration)/final total)*100
#((3729.000000/1)/18381.996078)*100
#Vtm=((Gross loss/1)/(Final total - class final total))*100
#((1.043251e+06/1)/(6.427637e+07-2.677706e+07))*100
#Rtin=((matrix)/duration)/initial total)*100
#((3.135244e+06/1)/8.137387e+06)*100
#Wtn=((Gross gain/1)/(Initial total - class initial total))*100
#((4.356936e+06/1)/(6.427637e+07-2.017387e+07))*100

#### 3.5.3. "Transition Intensity" calculation for Rtin, Wtn -> Gain, transition to target category from all other categories.

In [ ]:
# Initialize dictionary to hold category transition intensities for the significant intervals
category_transitions = {}

for year in significant_intervals:
    matrix = transition_matrices[year]

    # Access initial totals from the column directly
    initial_totals = matrix['Initial total'][:-2]  # Excluding the last row which is 'Gross gain'
    total_area = initial_totals.sum()

    # Create DataFrame to store transition intensities
    # Exclude 'Initial total' and 'Gross loss' from columns, and 'Final total' and 'Gross gain' from rows
    transition_intensities = pd.DataFrame(0, index=matrix.index[:-2], columns=matrix.columns[:-2].append(pd.Index(['Uniform_Intensity'])))

    for category_from in matrix.index[:-2]:  # Excluding 'Final total' and 'Gross gain' rows
        gross_gain = matrix.loc['Gross gain', category_from]
        area_not_n = total_area - matrix.loc[category_from, 'Initial total']
        
        if area_not_n > 0:
            uniform_intensity = (gross_gain / duration_of_interval) / area_not_n * 100
            transition_intensities.loc[category_from, 'Uniform_Intensity'] = uniform_intensity

        for category_to in matrix.columns[:-2]:  # Excluding 'Initial total' and 'Gross loss'
            if category_from != category_to:     # Ensure transitions between different categories
                transition_area = matrix.loc[category_from, category_to]
                initial_total_category_from = matrix.loc[category_from, 'Initial total']  # Using the source category's initial total
        
                if initial_total_category_from > 0:
                    transition_intensity = (transition_area / duration_of_interval) / initial_total_category_from * 100
                    transition_intensities.at[category_from, category_to] = transition_intensity

    # Extract the 'Uniform_Intensity' column as a separate Series
    uniform_intensity_row = transition_intensities['Uniform_Intensity'].copy()

    # Remove the 'Uniform_Intensity' column from its original place
    transition_intensities.drop('Uniform_Intensity', axis=1, inplace=True)

    # Append the 'Uniform_Intensity' Series as a row to the DataFrame
    transition_intensities.loc['Uniform_Intensity'] = uniform_intensity_row

    # Store the transition intensities matrix in the dictionary
    category_transitions[year] = transition_intensities

#### 3.5.4. "Transition Intensity" plots for Rtin, Wtn -> Gain, transition to target category from all other categories.

In [ ]:
for year in significant_intervals:
    matrix_to_plot = category_transitions[year]
    
    # Rounding the matrix values
    matrix_to_plot = matrix_to_plot.round(0)
    
    # Update the mask for the new shape
    mask = np.zeros_like(matrix_to_plot, dtype=bool)
    np.fill_diagonal(mask, True)  # Continue to hide diagonal values

    # Setting the colors for the mask
    cmap = mpl.colormaps.get_cmap('Reds')
    cmap.set_bad("white")  # setting background for masked elements
    
    # Plot heatmap with updated settings
    plt.figure(figsize=(12, 9))
    ax = sns.heatmap(matrix_to_plot, annot=True, fmt=".0f", cmap=cmap, linewidths=.5, mask=mask)
    
    # Add horizontal lines
    for i in range(matrix_to_plot.shape[0] + 1):  # adding lines at each row edge
        plt.axhline(y=i, color='black', linestyle='-', linewidth=1)
    
    plt.title(f"LULC transition intensity analysis for {Park}'s {inv_sub_area} buffer zone from {year}", fontsize=14)
    plt.xlabel('To Category (percent of category)')
    plt.xticks(rotation=45, ha="right")
    plt.ylabel('From Category & Uniform Intensity (percent of category)')
    
    plt.show()

In [ ]:
# This is purely for writing purposes (can copy this output into thesis)
# Prepare an empty list to collect data for transition intensities
#transition_summary_data = []

# Loop over each significant interval to analyze transition intensities
#for interval in significant_intervals:
    # Access the precomputed transition intensities matrix for this interval
#    transition_matrix = category_transitions[interval]
#    uniform_intensities = transition_matrix['Uniform_Intensity']

    # Loop through each 'from' category (exclude 'Uniform_Intensity' from columns)
#    for from_category in transition_matrix.index:
#        uniform_intensity = uniform_intensities[from_category]

        # Loop through each 'to' category
#        for to_category in transition_matrix.columns[:-1]:  # Exclude the last column which is 'Uniform_Intensity'
#            if from_category != to_category:  # Ensure we're looking at transitions between different categories
#                transition_intensity = transition_matrix.at[from_category, to_category]

                # Check if the transition intensity is greater than the uniform intensity
#                if transition_intensity > uniform_intensity:
#                    transition_summary_data.append({
#                        'Interval': interval,
#                        'From Category': from_category,
#                        'To Category': to_category,
#                        'Transition Intensity (%)': transition_intensity,
#                        'Uniform Intensity (%)': uniform_intensity
#                    })

# Create DataFrame from collected data
#transition_summary_df = pd.DataFrame(transition_summary_data)

# Set the display options to show all rows if desired
#pd.set_option('display.max_rows', None)

# Display the DataFrame
#display(transition_summary_df)

# 4. Plot the change hotspots for two years

### 4.1. Choose year interval to investigate

In [ ]:
year_opions = significant_intervals + ['Choose here...']

# Define the Dropdowns for selecting the starting and ending years
pre_post_year_dropdown = widgets.Dropdown(
    options=year_opions,
    value='Choose here...',
    description='Year interval of interest:',
    disabled=False,
)

# Initialize Variables
pre_post_year = pre_post_year_dropdown.value

# Define Update Functions for each variable
def on_pre_post_year_change(change):
    global pre_post_year
    pre_post_year = change.new
    print(f"Pre- and Post year of investigation updated to: {pre_post_year}")


# Attach the Update Functions to the respective Dropdowns
pre_post_year_dropdown.observe(on_pre_post_year_change, names='value')

# To display the dropdown in a Jupyter notebook
display(pre_post_year_dropdown)

#Note on how to use the multiple select tool
print('Note: To effectively use the widget below use "ctrl/cmd" on the kyboard to select multiple')

# Create a multi-select widget for the transition labels
select_widget = widgets.SelectMultiple(
    options=list(transition_label_map.values()),  # Display labels for selection
    value=[],  # Default value, no selection
    description='Active Changes',
    disabled=False,
    layout=Layout(width='350px', height='300px')
)

display(select_widget)

### 4.2. Set variables based on selections above

In [ ]:
# Change the format of the years based on selection above
inv_year_range = pre_post_year.replace('_to_', '-')

# Example to map selected labels back to their keys
selected_labels = select_widget.value  # This will be a tuple of selected label strings
selected_keys = [key for key, value in transition_label_map.items() if value in selected_labels]

# Convert selected keys to integers, as they are stored as strings in transition_label_map
transitions_of_interest = list(map(int, selected_keys))

### 4.3. Show the spatial extent of these changes on the map

In [ ]:
# Defining 'combined' for the map
combined = results_per_area_and_year_pairs[inv_sub_area][inv_year_range]

# Start with a condition that's always False
mask = ee.Image(0)

# Dynamically update the mask based on selected transitions
for transition in transitions_of_interest:
    mask = mask.Or(combined.eq(transition))

filtered_transitions = combined.updateMask(mask)

# Define visualization parameters with all red colors as per your request
vis_params = {
    'min': 0,
    'max': max(transitions_of_interest),  # Adjust max based on your data
    'palette': ['red'] * len(transitions_of_interest)
}

# To initiate map
Map = geemap.Map(basemap='Esri.WorldImagery')

# Use geemap to display the map with filtered transitions
Map.addLayer(filtered_transitions, vis_params, 'Filtered Transitions')

# To initiate display of map
display(Map)

In [ ]:
############################################## END ###################################################